In [1]:
!pip install nvcc4jupyter

In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpsap8uogm".


In [3]:
%%cuda
#include <iostream>
int main()
{
    std::cout << "C++ Working\n";
    return 0;
}

C++ Working



In [8]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>
#include <sys/time.h>
#include <iostream>
#include <cmath>
using namespace std;

#define SIZE 393216
#define SECOND_TO_MRCROSECOND (1000000)
#define NUM_BIN 768
#define ARRAYSIZE 256

__global__ void ecualizarImagen(int *d_a, int *d_acum) {
    // Calcula el índice global del hilo
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    if(idx <= ARRAYSIZE)
    {
      // Normaliza la función de acumulación
      int item = d_acum[idx];
      //printf("Acumulado en posicion %d: %d \n", idx, item);
      int result = (item*(ARRAYSIZE-1)) / SIZE;
      //printf("Resultado Normalizado para %d: %d \n",idx, result);
      d_a[idx] = result;
    }
}

int main()
{
	int h_a[ARRAYSIZE]{ 2084,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,1,4137,0,2,1,1,0,2,5455,0,0,3,2,4,4845,5,7,0,6,4601,6,0,10,4728,18,0,5617,85,234,0,6261,886,887,5719,738,511,5715,458,450,6432,0,6130,231,100,5754,58,6259,23,5772,11,5526,20,5696,22,5013,0,6278,20,4850,0,5515,20,5054,0,4965,4523,77,5046,60,4345,4977,56,4818,4373,0,4648,4491,59,4024,7784,226,4004,3603,3834,101,3459,2923,2858,19,2921,2742,2462,2392,0,2409,2116,2071,1946,2242,2287,1952,0,2113,1722,1888,2181,2051,1952,1762,2741,2260,2121,2337,2659,2814,2117,2512,3147,1849,2636,2450,2091,1563,2251,2864,1884,1152,1109,1569,832,1239,2018,1240,908,877,1150,1719,869,756,1036,1467,757,755,903,1405,506,730,1284,688,558,1166,713,419,1046,574,946,426,558,906,489,818,415,936,462,811,421,784,391,789,446,776,316,754,460,684,719,376,717,380,702,667,406,672,705,369,706,763,804,325,803,850,382,937,965,943,1084,444,1226,1120,1225,1367,1838,670,1656,2066,2403,2098,1952,2921,2773,2323,2334,1929,2290,1534,2004,1812,1454,1083,1095,909,762,611,685,558,499,506,556,349,593,482,612,614,1006,1502,8001 };
	float h_b[ARRAYSIZE];
  int h_acum[ARRAYSIZE];
	for(int i = 0; i<ARRAYSIZE; ++i)
  {
		h_b[i] = h_a[i];
    if (i == 0)
        h_acum[i] = h_a[i];
    else
        h_acum[i] = h_acum[i-1] + h_a[i];
	}

  std::cout << "Starting declaration for device variables" << std::endl;

  std::cout << "Creating acumulate array" << std::endl;
  std::cout << "\n[";
  for (int i = 0; i < ARRAYSIZE; i++)
  {
      //printf("bin %d : count %d\n", i, h_acum[i]);
      std::cout << h_acum[i] << ", ";
  }
  std::cout << "]";
  std::cout << "Starting process" << std::endl;
	// declare GPU memory pointers
	int *d_a;
	float *d_b;
  int *d_acum;

	// allocate GPU memory
	cudaMalloc((void **)&d_a, ARRAYSIZE*sizeof(int));
	//cudaMalloc((void **)&d_b, ARRAYSIZE*sizeof(float));
  cudaMalloc((void **)&d_acum, ARRAYSIZE*sizeof(int));

	// transfer the arrays to the GPU
	cudaMemcpy(d_a, h_a, ARRAYSIZE*sizeof(int), cudaMemcpyHostToDevice);
	//cudaMemcpy(d_b, h_b, ARRAYSIZE*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_acum, h_acum, ARRAYSIZE*sizeof(int), cudaMemcpyHostToDevice);

	//launch the kernel

  ecualizarImagen <<<1,ARRAYSIZE>>>(d_a, d_acum);
	//histogram_atomic <<<((SIZE+NUM_BIN-1)/NUM_BIN), 1>>>(d_b, d_a);
  std::cout << "Process finished" << std::endl << std::endl;
	// copy back the sum from GPU
	cudaMemcpy(h_a, d_a, ARRAYSIZE*sizeof(int), cudaMemcpyDeviceToHost);

  std::cout << "Equalized Histogram" << std::endl << std::endl;
  std::cout << "\n[";
  for (int i = 0; i < ARRAYSIZE; i++)
  {

      std::cout << h_a[i] << ", ";
  }
  std::cout << "]";

	// free GPU memory allocation
	cudaFree(d_a);
	//cudaFree(d_b);
  cudaFree(d_acum);

	return 0;

}

Starting declaration for device variables
Creating acumulate array

[2084, 2084, 2084, 2084, 2084, 2084, 2084, 2084, 2084, 2084, 2084, 2084, 2085, 2085, 2085, 2085, 2087, 2089, 2089, 2089, 2090, 6227, 6227, 6229, 6230, 6231, 6231, 6233, 11688, 11688, 11688, 11691, 11693, 11697, 16542, 16547, 16554, 16554, 16560, 21161, 21167, 21167, 21177, 25905, 25923, 25923, 31540, 31625, 31859, 31859, 38120, 39006, 39893, 45612, 46350, 46861, 52576, 53034, 53484, 59916, 59916, 66046, 66277, 66377, 72131, 72189, 78448, 78471, 84243, 84254, 89780, 89800, 95496, 95518, 100531, 100531, 106809, 106829, 111679, 111679, 117194, 117214, 122268, 122268, 127233, 131756, 131833, 136879, 136939, 141284, 146261, 146317, 151135, 155508, 155508, 160156, 164647, 164706, 168730, 176514, 176740, 180744, 184347, 188181, 188282, 191741, 194664, 197522, 197541, 200462, 203204, 205666, 208058, 208058, 210467, 212583, 214654, 216600, 218842, 221129, 223081, 223081, 225194, 226916, 228804, 230985, 233036, 234988, 236750, 2